In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    fbeta_score,
    make_scorer,
    log_loss,
)
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px
from pathlib import Path
from tqdm.notebook import tqdm

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from joblib import parallel_backend
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score

In [2]:
# NUM_POSTE   : numéro Météo-France du poste sur 8 chiffres
# NOM_USUEL   : nom usuel du poste
# LAT         : latitude, négative au sud (en degrés et millionièmes de degré)
# LON         : longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        : altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJ    : date de la mesure (année mois jour)
# RR          : quantité de précipitation tombée en 24 heures (de 06h FU le jour J à 06h FU le jour J+1). La valeur relevée à J+1 est affectée au jour J (en mm et 1/10)
# TN          : température minimale sous abri (en °C et 1/10)
# HTN         : heure de TN (hhmm)
# TX          : température maximale sous abri (en °C et 1/10)
# HTX         : heure de TX (hhmm)
# TM          : moyenne quotidienne des températures horaires sous abri (en °C et 1/10)
# TNTXM       : moyenne quotidienne (TN+TX)/2 (en °C et 1/10)
# TAMPLI      : amplitude thermique quotidienne : écart entre TX et TN quotidiens (TX-TN) (en °C et 1/10)
# TNSOL       : température quotidienne minimale à 10 cm au-dessus du sol (en °C et 1/10)
# TN50        : température quotidienne minimale à 50 cm au-dessus du sol (en °C et 1/10)
# DG          : durée de gel sous abri (T ≤ 0°C) (en mn)
# FFM         : moyenne quotidienne de la force du vent moyenné sur 10 mn, à 10 m (en m/s et 1/10)
# FF2M        : moyenne quotidienne de la force du vent moyenné sur 10 mn, à 2 m (en m/s et 1/10)
# FXY         : maximum quotidien de la force maximale horaire du vent moyenné sur 10 mn, à 10 m (en m/s et 1/10)
# DXY         : direction de FXY (en rose de 360)
# HXY         : heure de FXY (hhmm)
# FXI         : maximum quotidien de la force maximale horaire du vent instantané, à 10 m (en m/s et 1/10)
# DXI         : direction de FXI (en rose de 360)
# HXI         : heure de FXI (hhmm)
# FXI2        : maximum quotidien de la force maximale horaire du vent instantané, à 2 m (en m/s et 1/10)
# DXI2        : direction de FXI2 (en rose de 360)
# HXI2        : heure de FXI2 (hhmm)
# FXI3S       : maximum quotidien de la force maximale horaire du vent moyenné sur 3 s, à 10 m (en m/s et 1/10)
# DXI3S       : direction de FXI3S (en rose de 360)
# HXI3S       : heure de FXI3S (hhmm)
# DRR         : durée des précipitations (en mn)

# A chaque donnée est associé un code qualité (ex: T;QT) :
#  9 : donnée filtrée (la donnée a passé les filtres/contrôles de premiers niveaux)
#  0 : donnée protégée (la donnée a été validée définitivement par le climatologue)
#  1 : donnée validée (la donnée a été validée par contrôle automatique ou par le climatologue)
#  2 : donnée douteuse en cours de vérification (la donnée a été mise en doute par contrôle automatique)

# D'une façon générale, les valeurs fournies sont données avec une précision qui correspond globalement à la résolution de l'appareil de mesure de la valeur.
# Toutefois, il peut arriver, pour des raisons techniques de stokage ou d'extraction des valeurs, que cette règle ne soit pas respectée.
# Du fait d'arrondis, il peut ponctuellement arriver que des valeurs de base à un pas de temps inférieur (par exemple données minutes) ne soient pas exactement cohérentes avec leurs correspondants sur un pas de temps supérieur (par exemple données horaires).

In [3]:
df = pd.read_feather("../data/merged_meteo_red_days_from_20170101.feather")
print(df.columns)
df.head()

Index(['AAAAMMJJ', 'TN_BORDEAUX', 'TN_LILLE', 'TN_LYON', 'TN_MARSEILLE',
       'TN_MONTPELLIER', 'TN_NANTES', 'TN_NICE', 'TN_PARIS', 'TN_STRASBOURG',
       'TN_TOULOUSE', 'TX_BORDEAUX', 'TX_LILLE', 'TX_LYON', 'TX_MARSEILLE',
       'TX_MONTPELLIER', 'TX_NANTES', 'TX_NICE', 'TX_PARIS', 'TX_STRASBOURG',
       'TX_TOULOUSE', 'TM_BORDEAUX', 'TM_LILLE', 'TM_LYON', 'TM_MARSEILLE',
       'TM_MONTPELLIER', 'TM_NANTES', 'TM_NICE', 'TM_PARIS', 'TM_STRASBOURG',
       'TM_TOULOUSE', 'TNTXM_BORDEAUX', 'TNTXM_LILLE', 'TNTXM_LYON',
       'TNTXM_MARSEILLE', 'TNTXM_MONTPELLIER', 'TNTXM_NANTES', 'TNTXM_NICE',
       'TNTXM_PARIS', 'TNTXM_STRASBOURG', 'TNTXM_TOULOUSE', 'TAMPLI_BORDEAUX',
       'TAMPLI_LILLE', 'TAMPLI_LYON', 'TAMPLI_MARSEILLE', 'TAMPLI_MONTPELLIER',
       'TAMPLI_NANTES', 'TAMPLI_NICE', 'TAMPLI_PARIS', 'TAMPLI_STRASBOURG',
       'TAMPLI_TOULOUSE', 'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_LYON',
       'FFM_MARSEILLE', 'FFM_MONTPELLIER', 'FFM_NANTES', 'FFM_NICE',
       'FFM_PARIS', 'FFM

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,...,RR_LILLE,RR_LYON,RR_MARSEILLE,RR_MONTPELLIER,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day
0,20170101,2.6,-4.3,-1.2,4.5,2.4,-4.0,4.6,-4.3,-2.9,...,1.4,0.0,0.2,0.0,1.6,0.0,0.2,0.0,0.0,False
1,20170102,2.2,-1.0,-0.5,6.3,3.3,3.8,5.6,-0.6,-1.9,...,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.0,0.0,False
2,20170103,1.3,-2.6,-0.8,-0.1,0.3,0.0,5.1,0.9,-3.7,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,20170104,-4.9,2.1,-2.0,1.5,-2.8,-3.2,5.4,0.1,-1.9,...,2.4,1.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,True
4,20170105,-2.8,0.3,0.3,2.4,1.6,0.1,4.4,2.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [4]:
# Drop non TN/TX/TM cols
# cols_to_keep = [col for col in df.columns if  'TN_' in col or  'TX_' in col in col or col == 'AAAAMMJJ' or col == 'is_red_day']
# df = df[cols_to_keep]

In [5]:
# Remove days after 20240311
df = df[df["AAAAMMJJ"] < 20240401]
df.tail()

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,...,RR_LILLE,RR_LYON,RR_MARSEILLE,RR_MONTPELLIER,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day
2622,20240307,5.7,NaN,0.3,0.8,0.5,4.0,7.4,3.8,3.6,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2623,20240308,8.9,NaN,4.8,7.0,8.7,6.9,6.9,3.6,-0.8,...,NaN,0.4,8.0,13.1,1.4,8.1,0.0,0.0,12.9,False
2624,20240309,7.5,NaN,5.9,7.7,9.9,6.2,8.2,6.4,2.3,...,NaN,15.1,69.4,21.8,8.5,40.3,6.5,0.0,0.0,False
2625,20240310,3.3,NaN,5.7,9.2,6.3,0.9,8.4,7.1,6.4,...,NaN,4.4,3.6,0.0,7.5,23.8,0.0,7.3,0.2,False
2626,20240311,4.5,NaN,5.1,6.7,7.6,3.9,8.1,6.4,3.8,...,NaN,4.2,0.6,0.2,0.2,0.0,2.0,0.0,10.7,True


In [6]:
# Count nan values and show nan cols:

print(df.isna().sum())
pd.set_option("display.max_rows", None)
df.columns[df.isna().sum() > 0]

AAAAMMJJ          0
TN_BORDEAUX       0
TN_LILLE         71
TN_LYON           0
TN_MARSEILLE      0
                 ..
RR_NICE           0
RR_PARIS          0
RR_STRASBOURG     0
RR_TOULOUSE       0
is_red_day        0
Length: 72, dtype: int64


Index(['TN_LILLE', 'TN_MONTPELLIER', 'TX_LILLE', 'TX_MONTPELLIER', 'TM_LILLE',
       'TM_MARSEILLE', 'TM_MONTPELLIER', 'TM_NICE', 'TNTXM_LILLE',
       'TNTXM_MONTPELLIER', 'TAMPLI_LILLE', 'TAMPLI_MONTPELLIER',
       'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_MARSEILLE', 'FFM_MONTPELLIER',
       'FFM_PARIS', 'FFM_STRASBOURG', 'FFM_TOULOUSE', 'RR_LILLE',
       'RR_MONTPELLIER'],
      dtype='object')

In [7]:
df.columns

Index(['AAAAMMJJ', 'TN_BORDEAUX', 'TN_LILLE', 'TN_LYON', 'TN_MARSEILLE',
       'TN_MONTPELLIER', 'TN_NANTES', 'TN_NICE', 'TN_PARIS', 'TN_STRASBOURG',
       'TN_TOULOUSE', 'TX_BORDEAUX', 'TX_LILLE', 'TX_LYON', 'TX_MARSEILLE',
       'TX_MONTPELLIER', 'TX_NANTES', 'TX_NICE', 'TX_PARIS', 'TX_STRASBOURG',
       'TX_TOULOUSE', 'TM_BORDEAUX', 'TM_LILLE', 'TM_LYON', 'TM_MARSEILLE',
       'TM_MONTPELLIER', 'TM_NANTES', 'TM_NICE', 'TM_PARIS', 'TM_STRASBOURG',
       'TM_TOULOUSE', 'TNTXM_BORDEAUX', 'TNTXM_LILLE', 'TNTXM_LYON',
       'TNTXM_MARSEILLE', 'TNTXM_MONTPELLIER', 'TNTXM_NANTES', 'TNTXM_NICE',
       'TNTXM_PARIS', 'TNTXM_STRASBOURG', 'TNTXM_TOULOUSE', 'TAMPLI_BORDEAUX',
       'TAMPLI_LILLE', 'TAMPLI_LYON', 'TAMPLI_MARSEILLE', 'TAMPLI_MONTPELLIER',
       'TAMPLI_NANTES', 'TAMPLI_NICE', 'TAMPLI_PARIS', 'TAMPLI_STRASBOURG',
       'TAMPLI_TOULOUSE', 'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_LYON',
       'FFM_MARSEILLE', 'FFM_MONTPELLIER', 'FFM_NANTES', 'FFM_NICE',
       'FFM_PARIS', 'FFM

In [8]:
# drop montepellier
df.drop(columns=[c for c in df.columns if "MONTPELLIER" in c], axis=1, inplace=True)

In [9]:
print(df.isna().sum())

AAAAMMJJ              0
TN_BORDEAUX           0
TN_LILLE             71
TN_LYON               0
TN_MARSEILLE          0
TN_NANTES             0
TN_NICE               0
TN_PARIS              0
TN_STRASBOURG         0
TN_TOULOUSE           0
TX_BORDEAUX           0
TX_LILLE             71
TX_LYON               0
TX_MARSEILLE          0
TX_NANTES             0
TX_NICE               0
TX_PARIS              0
TX_STRASBOURG         0
TX_TOULOUSE           0
TM_BORDEAUX           0
TM_LILLE             71
TM_LYON               0
TM_MARSEILLE          5
TM_NANTES             0
TM_NICE               1
TM_PARIS              0
TM_STRASBOURG         0
TM_TOULOUSE           0
TNTXM_BORDEAUX        0
TNTXM_LILLE          71
TNTXM_LYON            0
TNTXM_MARSEILLE       0
TNTXM_NANTES          0
TNTXM_NICE            0
TNTXM_PARIS           0
TNTXM_STRASBOURG      0
TNTXM_TOULOUSE        0
TAMPLI_BORDEAUX       0
TAMPLI_LILLE         71
TAMPLI_LYON           0
TAMPLI_MARSEILLE      0
TAMPLI_NANTES   

In [10]:
df.dropna(inplace=True)

Feature engineering

In [11]:
# Add a is week day feature:
daydt = pd.to_datetime(df["AAAAMMJJ"], format="%Y%m%d").dt
df["is_week_day"] = daydt.dayofweek < 5

In [12]:
# Add the number of elapsed red days for the current year
from datetime import datetime

df["is_red_day"] = df["is_red_day"].astype(int)

# df['elapsed_red_days'] = df.groupby(daydt.year)['is_red_day'].cumsum()
# df['fiscal_year'] = (daydt.year + (daydt.month >= 4 ).astype(int)).astype(int)
# df['remaining_red_days'] = 22 - df.groupby('fiscal_year')['is_red_day'].shift(1).fillna(0).cumsum().astype(int)
# df.drop(columns=['fiscal_year'], inplace=True)

df["last_day_was_red"] = df["is_red_day"].shift(1).fillna(0).astype(int)

In [13]:
# Remove data between  01/04 and 01/11 and prior to 2017-04
df = df[~((daydt.month >= 4) & (daydt.month <= 10))]
df = df[~((daydt.year == 2017) & (daydt.month < 4))]

C:\Users\Fabien\AppData\Local\Temp\ipykernel_2204\438820387.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~((daydt.year == 2017) & (daydt.month < 4))]


In [14]:
df.columns

Index(['AAAAMMJJ', 'TN_BORDEAUX', 'TN_LILLE', 'TN_LYON', 'TN_MARSEILLE',
       'TN_NANTES', 'TN_NICE', 'TN_PARIS', 'TN_STRASBOURG', 'TN_TOULOUSE',
       'TX_BORDEAUX', 'TX_LILLE', 'TX_LYON', 'TX_MARSEILLE', 'TX_NANTES',
       'TX_NICE', 'TX_PARIS', 'TX_STRASBOURG', 'TX_TOULOUSE', 'TM_BORDEAUX',
       'TM_LILLE', 'TM_LYON', 'TM_MARSEILLE', 'TM_NANTES', 'TM_NICE',
       'TM_PARIS', 'TM_STRASBOURG', 'TM_TOULOUSE', 'TNTXM_BORDEAUX',
       'TNTXM_LILLE', 'TNTXM_LYON', 'TNTXM_MARSEILLE', 'TNTXM_NANTES',
       'TNTXM_NICE', 'TNTXM_PARIS', 'TNTXM_STRASBOURG', 'TNTXM_TOULOUSE',
       'TAMPLI_BORDEAUX', 'TAMPLI_LILLE', 'TAMPLI_LYON', 'TAMPLI_MARSEILLE',
       'TAMPLI_NANTES', 'TAMPLI_NICE', 'TAMPLI_PARIS', 'TAMPLI_STRASBOURG',
       'TAMPLI_TOULOUSE', 'FFM_BORDEAUX', 'FFM_LILLE', 'FFM_LYON',
       'FFM_MARSEILLE', 'FFM_NANTES', 'FFM_NICE', 'FFM_PARIS',
       'FFM_STRASBOURG', 'FFM_TOULOUSE', 'RR_BORDEAUX', 'RR_LILLE', 'RR_LYON',
       'RR_MARSEILLE', 'RR_NANTES', 'RR_NICE', 'RR_PARI

Learning

In [15]:
#  show values count in y
df["is_red_day"].value_counts()

is_red_day
0    827
1    129
Name: count, dtype: int64

In [16]:
def evaluate(model, X, y, plot_roc_rurve=False):
    y_pred = model.predict(X)
    print(f"Accuracy: {accuracy_score(y, y_pred)}")
    print("Precision", precision_score(y, y_pred))
    print("Recall", recall_score(y, y_pred))
    print("F1 score", f1_score(y, y_pred))
    print("F2 score", fbeta_score(y, y_pred, beta=2))
    display(confusion_table(y_pred, y))

    if plot_roc_rurve:
        y_pred_proba = model.predict_proba(X)[:, 1]
        fpr, tpr, _ = roc_curve(y, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        fig = px.area(
            x=fpr,
            y=tpr,
            title=f"ROC Curve (AUC={roc_auc:.4f})",
            labels=dict(x="False Positive Rate", y="True Positive Rate"),
            width=700,
            height=500,
        )
        fig.show()

In [17]:
X = df.drop(["is_red_day", "AAAAMMJJ"], axis=1)
todrop = []
for c in X.columns:
    if "TAMPLI" in c or "RR" in c or "FFM" in c or "TM" in c or "TNTXM" in c:
        todrop.append(c)


def encode_categorical(df, categorical_cols):
    for c in categorical_cols:
        if c in df.columns:
            df[c] = df[c].astype("category")
    return df


X = X.drop(todrop, axis=1)
encode_categorical(X, ["is_week_day", "last_day_was_red"])
y = df["is_red_day"]


X_train, X_test, y_train, y_test = skm.train_test_split(
    X, y, test_size=0.2, random_state=42
)
f2_scorer = make_scorer(fbeta_score, beta=2)

In [18]:
X.columns

Index(['TN_BORDEAUX', 'TN_LILLE', 'TN_LYON', 'TN_MARSEILLE', 'TN_NANTES',
       'TN_NICE', 'TN_PARIS', 'TN_STRASBOURG', 'TN_TOULOUSE', 'TX_BORDEAUX',
       'TX_LILLE', 'TX_LYON', 'TX_MARSEILLE', 'TX_NANTES', 'TX_NICE',
       'TX_PARIS', 'TX_STRASBOURG', 'TX_TOULOUSE', 'is_week_day',
       'last_day_was_red'],
      dtype='object')

In [19]:
# # Define the parameter grid
# param_grid = {
#     "n_estimators": [50, 100, 200],
#     "learning_rate": [0.01, 0.05, 0.1],
#     "max_depth": [5, 10, 20],
#     "num_leaves": [20, 50, 100],
#     "min_child_samples": [10, 20, 50],
#     "bagging_fraction": [0.6, 0.8],
#     "feature_fraction": [0.6, 0.8],
#     "reg_alpha": [0, 0.1, 1],
#     "reg_lambda": [0, 0.1, 1],
#     "subsample": [0.7, 1.0],
#     "colsample_bytree": [0.7, 1.0],
#     "scale_pos_weight": [10, 20],
# }

# # Perform GridSearchCV
# lgbm = LGBMClassifier(random_state=0, force_row_wise=True, verbose=-1, n_jobs=-1)

# kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# random_search = RandomizedSearchCV(
#     lgbm, param_grid,
#     scoring=f2_scorer,
#     n_iter=1000,
#     cv=kf,
#     verbose=1,
#     n_jobs=-1,
#     random_state=42,
# )

# with parallel_backend('multiprocessing'):
#     random_search.fit(X_train, y_train)

# # Print the best parameters and evaluate
# print("Best parameters found: ", random_search.best_params_)
# print('Best F2 score:', random_search.best_score_)

In [20]:
# Obtained with random search
best_params = {
    "subsample": 1.0,
    "scale_pos_weight": 10,
    "reg_lambda": 1,
    "reg_alpha": 1,
    "num_leaves": 20,
    "n_estimators": 50,
    "min_child_samples": 50,
    "max_depth": 5,
    "learning_rate": 0.05,
    "feature_fraction": 0.8,
    "colsample_bytree": 0.7,
    "bagging_fraction": 0.6,
}
lgbm = LGBMClassifier(random_state=0, force_row_wise=True, verbose=-1, **best_params)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f2_scorer = make_scorer(fbeta_score, beta=2)

score = cross_val_score(lgbm, X_train, y_train, cv=kf, scoring=f2_scorer)
print("Cross Validation F2 scores are: {}".format(score))
print("Average Cross Validation F2 score: {}".format(score.mean()))

lgbm.fit(X_train, y_train)
evaluate(lgbm, X_train, y_train)
evaluate(lgbm, X_test, y_test)

Cross Validation F2 scores are: [0.74074074 0.77669903 0.66964286 0.75892857 0.69672131]
Average Cross Validation F2 score: 0.7285465019827585
Accuracy: 0.912303664921466
Precision 0.6049382716049383
Recall 0.9702970297029703
F1 score 0.7452471482889734
F2 score 0.8657243816254417


Truth,0,1
Predicted,,
0,599,3
1,64,98


Accuracy: 0.8541666666666666
Precision 0.5
Recall 0.8214285714285714
F1 score 0.6216216216216216
F2 score 0.7278481012658228


Truth,0,1
Predicted,,
0,141,5
1,23,23


Explore incorrect preds

In [37]:
# Explore incorrect predictions

y_pred = lgbm.predict(X_test)
X_test_indices = X_test.index
df_test = df.loc[X_test_indices].copy()
df_test["y_pred"] = y_pred

incorrect_preds = df_test[df_test["is_red_day"] != df_test["y_pred"]]

assert all(
    incorrect_preds["y_pred"] != incorrect_preds["is_red_day"]
), "y_pred should be different from is_red_day"
incorrect_preds[
    (incorrect_preds["is_red_day"] == 0) | (incorrect_preds["y_pred"] == 1)
].head(15)

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,TN_TOULOUSE,...,RR_MARSEILLE,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day,is_week_day,last_day_was_red,y_pred
1849,20220124,-1.3,0.2,-2.8,0.7,0.5,4.9,0.7,-2.0,2.1,...,0.2,0.0,0.0,0.0,0.0,0.0,0,True,0,1
443,20180320,1.3,-2.1,0.6,2.0,-0.2,6.2,-1.3,-2.8,2.4,...,0.0,0.0,0.4,0.0,0.0,0.0,0,True,0,1
334,20171201,0.3,-0.7,0.0,-0.4,-0.5,4.6,0.8,0.1,1.9,...,0.2,0.2,8.8,0.0,0.8,0.6,0,True,0,1
2220,20230130,-2.8,3.7,-1.0,-2.2,3.8,3.1,4.0,-4.4,2.2,...,0.0,0.8,0.0,0.4,0.2,0.0,0,True,0,1
1908,20220324,4.7,3.5,3.1,2.5,2.9,9.0,8.4,0.1,5.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0,True,1,1
1543,20210324,1.1,2.3,2.2,3.7,-0.8,7.5,4.1,-2.2,1.8,...,0.2,0.2,0.0,1.6,0.0,0.0,0,True,1,1
2174,20221215,8.1,-6.2,3.8,9.1,1.2,7.4,-2.0,-2.4,8.7,...,14.4,0.6,50.3,0.0,0.4,3.6,0,True,1,1
787,20190227,6.1,3.8,1.7,4.9,4.6,9.5,5.4,-1.4,2.9,...,0.2,0.2,0.0,0.0,0.0,0.0,0,True,1,1
1816,20211222,1.5,-3.7,-3.9,2.2,-1.8,7.9,-1.5,-3.7,4.2,...,0.2,0.0,0.0,0.0,0.0,0.0,0,True,0,1
1160,20200306,6.7,4.8,4.7,4.4,4.3,5.8,6.0,6.1,4.3,...,0.0,5.9,0.0,0.0,0.4,8.0,0,True,1,1


Export to onnx

In [44]:
import onnxmltools
from onnxmltools.convert import convert_lightgbm
import onnx
from onnxmltools.convert.common.data_types import FloatTensorType
from onnxsim import simplify
from onnx import helper

X_train_np = X_train.to_numpy().astype(np.float32)
initial_types = [("input", FloatTensorType([None, X_train_np.shape[1]]))]
input_names_str = ",".join(X_train.columns)

onnx_model = convert_lightgbm(
    lgbm,
    initial_types=initial_types,
    target_opset=12,
    zipmap=False,
    doc_string=f"Predict if the next day will be a red day.\n\nInput names: {input_names_str}",
)

# Simplify the ONNX model to avoid unsupported operators in app later on
simp_model, check = simplify(onnx_model)
assert check, "Simplified ONNX model could not be validated"

The maximum opset needed by this model is only 9.


In [45]:
# Now we will modify the onnx model to name the input features
model = simp_model

# Get the original input tensor
graph = model.graph
original_input = graph.input[0]  # Assuming there is only one input

# Extract input name, type, and shape
input_name = original_input.name
input_type = original_input.type.tensor_type.elem_type  # Data type
old_shape = original_input.type.tensor_type.shape.dim

# Check current shape (should be [None, 20])
print("Original Shape:", [dim.dim_value for dim in old_shape])

# Create 20 separate input tensors (None, 1)
features_names = X_train.columns
new_inputs = [
    helper.make_tensor_value_info(features_names[i], input_type, [None, 1])
    for i in range(len(features_names))
]

# Create a new node that concatenates inputs (needed for LightGBM)
concat_node = helper.make_node(
    "Concat",
    inputs=features_names,
    outputs=["concatenated_input"],
    axis=1,  # feature axis
)

# Replace old input with new inputs and modify the first node to use `concatenated_input`
graph.input.remove(original_input)
graph.input.extend(new_inputs)

# Find the first node that takes the original input and modify it
for node in graph.node:
    for i, input_name_in_node in enumerate(node.input):
        if input_name_in_node == input_name:
            node.input[i] = "concatenated_input"

# Add the new Concat node at the beginning
graph.node.insert(0, concat_node)

# Save the modified model
onnx_model_path = "lgbm_model_red_days_2025_02_28.onnx"
onnx.save(model, onnx_model_path)

print(f"Modified ONNX model saved as {onnx_model_path}")

Original Shape: [0, 20]
Modified ONNX model saved as lgbm_model_red_days_2025_02_28.onnx


In [46]:
# Load model to test it in python using onnxruntime:
import onnxruntime as rt

sess = rt.InferenceSession(onnx_model_path)
input_names = [i.name for i in sess.get_inputs()]
label_name = sess.get_outputs()[0].name

# Test the model
X_test_np = X_test.to_numpy().astype(np.float32)
input_data = {
    input_name: np.expand_dims(X_test_np[..., i], axis=-1)
    for i, input_name in enumerate(input_names)
}
pred_onnx = sess.run([label_name], input_data)[0]

# Compare the predictions
assert np.allclose(y_pred, pred_onnx), "Predictions should be the same"